<a href="https://colab.research.google.com/github/H-H-E/-/blob/main/360_Colab_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#360 Panorama Colab Demo

### by [Hussein](https://twitter.com/HHElafifi)

###based on original generation code by [angrypenguinPNG](https://twitter.com/angrypenguinPNG)


Run all should work.


#Install Requirements


In [23]:
#step 1
!pip install diffusers==0.19.1
!pip install transformers
!pip install accelerate
!pip install gradio
!pip install datasets
!pip install requests



#Import Requirements

In [24]:
import gradio as gr
from datasets import load_dataset
import requests

from PIL import Image
import re
import base64
from io import BytesIO
from diffusers import StableDiffusionLDM3DPipeline
import torch

# Load your custom StableDiffusionLDM3DPipeline
pipe = StableDiffusionLDM3DPipeline.from_pretrained("Intel/ldm3d-pano")
pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


StableDiffusionLDM3DPipeline {
  "_class_name": "StableDiffusionLDM3DPipeline",
  "_diffusers_version": "0.19.1",
  "_name_or_path": "Intel/ldm3d-pano",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "DDIMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

#Prepare a Pannellum HTML to inject into the Gradio UX

In [25]:
def create_pannellum_html(image_path):
    pannellum_html = f'''


<iframe width="600" height="400" allowfullscreen style="border-style:none;" src="https://cdn.pannellum.org/2.5/pannellum.htm#panorama={image_path}"></iframe>


    '''
    print(pannellum_html)
    return pannellum_html

#Upload image to imgbb
Gradio doesn't have access to locally (Colab lol) stored images so we have to upload it to imgbb for retreival via Panellum

In [26]:

def upload_to_imgbb(image, prompt, imgbb_api_key):
    # Convert the image to a binary file
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    image_binary = buffered.getvalue()

    # ImgBB API endpoint
    url = "https://api.imgbb.com/1/upload"

    # API parameters
    payload = {
        "key": imgbb_api_key,
        "image": base64.b64encode(image_binary)
    }

    # Make the POST request
    response = requests.post(url, payload)
    response_data = response.json()

    # Extract and return the URL of the uploaded image
    return response_data["data"]["url"]

#Combine

![](https://media.tenor.com/1kdTYqdDxi0AAAAC/combo-combination.gif)


In [27]:
def infer(prompt, negative, scale):
    # Generate the image
    output = pipe(prompt, width=1024, height=512, guidance_scale=scale, num_inference_steps=50)
    rgb_image, _ = output.rgb, output.depth

    # Upload the image to ImgBB and get the URL
    imgbb_api_key = 'cabf8280a38ee0969bbff05fe6060269'
    uploaded_image_url = upload_to_imgbb(rgb_image[0], prompt, imgbb_api_key)

    # Create the HTML using the uploaded image's URL
    return create_pannellum_html(uploaded_image_url)

#Run Gradio Interface


In [28]:

# Gradio interface setup
with gr.Blocks() as block:
    with gr.Row():
        text = gr.Textbox(label="Prompt", placeholder="Enter a prompt")
        negative = gr.Textbox(label="Negative Prompt", placeholder="Enter a negative prompt")
        scale = gr.Slider(minimum=1, maximum=10, label="Guidance Scale", value=7.5)
    btn = gr.Button("Generate")
    panorama_viewer = gr.HTML()  # HTML component for Pannellum

    btn.click(infer, inputs=[text, negative, scale], outputs=[panorama_viewer])

# Launch the Gradio interface
block.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://94ef96d26a40c79b4e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?it/s]


 

<iframe width="600" height="400" allowfullscreen style="border-style:none;" src="https://cdn.pannellum.org/2.5/pannellum.htm#panorama=https://i.ibb.co/5rq10HT/460804cefed4.jpg"></iframe>

    
    


  0%|          | 0/50 [00:00<?, ?it/s]


 

<iframe width="600" height="400" allowfullscreen style="border-style:none;" src="https://cdn.pannellum.org/2.5/pannellum.htm#panorama=https://i.ibb.co/DGzBsXY/ab94918eefa4.jpg"></iframe>

    
    
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://94ef96d26a40c79b4e.gradio.live
